In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the pivot DataFrame
pivot_df = pd.read_csv('simulated_sensor_data.csv')
#pivot_df = pivot_df.head(2000)

# Parameters
H = np.array([[1, 0]])                # Measurement matrix
Q = np.array([[1e-5, 0], [0, 1e-5]])  # Process noise covariance
R = np.array([[0.5]])                 # Measurement noise covariance
epsilon = 2                         # Sensitivity threshold for event detection
theta = 2                        # Threshold for significant change
num_nodes_to_poll_list = [20]         # List of numbers of nodes to poll
force_pull_threshold = 200            # Forcefully pull a node if it hasn't been pulled in the last 10 time steps
rho = 1                            # Probability of successful transmission for Whittle index

# Energy parameters in Joules
E_max = 162000  # Battery capacity in Joules
E_t = 50 / 1000  # Transmission energy in Joules
E_s = 10 / 1000  # Sensing energy in Joules
E_w = 10 / 1000  # Wake-up energy in Joules
E_0 = 1 / 1000   # Sleep energy in Joules

# Initialize previous timestamps for delta time calculation
last_update_times = {f'mote{i}': 0 for i in range(1, 51)}

# Initialize tracking arrays
polled_count = {f'mote{i}': 0 for i in range(1, 51)}
transmitted_count = {f'mote{i}': 0 for i in range(1, 51)}

# Initialize state estimates and covariance matrices for Kalman filter
state_estimates = {f'mote{i}': np.array([[20], [0.01]]) for i in range(1, 51)}
P = {f'mote{i}': np.zeros((2, 2)) for i in range(1, 51)}

def calculate_whittle_index(j, rho):
    return j * (j + 1) * rho / 2 

def get_state_transition_matrix(delta_t):
    return np.array([[1, delta_t], [0, 1]])

def predict_node_state(x_hat, delta_t):
    A_delta = get_state_transition_matrix(delta_t)  # Adjust the state transition matrix for the time step
    return A_delta @ x_hat

def run_simulation(num_nodes_to_poll):
    valuable_sensor_data = []
    transmission_events_1 = {f'mote{i}': [] for i in range(1, 51)}  # To track transmission events for time 1-1000
    transmission_events_2 = {f'mote{i}': [] for i in range(1, 51)}  # To track transmission events for time 1000-2000
    previously_polled_nodes = set()  # Initialize previously_polled_nodes
    mse_values = {f'mote{i}': [] for i in range(1, 51)}  # To track MSE values for each node

    for idx, row in pivot_df.iterrows():
        current_time_step = idx
        currently_polled_nodes = set()

        # Check if any node needs to be forcefully pulled due to inactivity
        for mote, last_time in last_update_times.items():
            if current_time_step - last_time >= force_pull_threshold:
                currently_polled_nodes.add(mote)

        # Calculate the combined metric for each node
        combined_metrics = {}
        for mote in last_update_times:
            j = current_time_step - last_update_times[mote]
            whittle_index = calculate_whittle_index(j, rho)
            delta_t = max(current_time_step - last_update_times[mote], 1)
            predicted_state = predict_node_state(state_estimates[mote], delta_t)
            predicted_value = H @ predicted_state
            current_value = row[mote]
            distance_metric = np.abs(predicted_value - current_value)
            combined_metrics[mote] = whittle_index * distance_metric

        # Select top M nodes based on the combined metric
        nodes_by_combined_metric = sorted(combined_metrics.keys(), key=lambda mote: combined_metrics[mote], reverse=True)
        top_m_nodes = nodes_by_combined_metric[:num_nodes_to_poll]

        # Include forced pull nodes in the top sensors list
        nodes_to_poll = list(currently_polled_nodes) + top_m_nodes

        for mote in nodes_to_poll:
            polled_count[mote] += 1
            transmitted_count[mote] += 1
            measured_value = row[mote]
            j = current_time_step - last_update_times[mote]

            previous_state = state_estimates[mote]
            previous_P = P[mote]
            delta_t = max(current_time_step - last_update_times[mote], 1)
            A_delta = get_state_transition_matrix(delta_t)  # Get the dynamic state transition matrix

            xp = A_delta @ previous_state
            Pp = A_delta @ previous_P @ A_delta.T + Q

            z = np.array([[measured_value]])
            K = Pp @ H.T @ np.linalg.inv(H @ Pp @ H.T + R)
            x_hat = xp + K @ (z - H @ xp)
            P_hat = Pp - K @ H @ Pp

            state_estimates[mote] = x_hat
            P[mote] = P_hat

            predicted_measurement = predict_node_state(previous_state, delta_t)[0, 0]
            diff = abs(predicted_measurement - measured_value)
            mse_values[mote].append(diff**2)

            valuable_sensor_data.append({
                'index': current_time_step,
                'selected_moteid': mote,
                'temperature': measured_value,
                'time_elapsed': j
            })

            # Track the transmission event for the two periods
            if 0 <= current_time_step < 1000:
                transmission_events_1[mote].append(current_time_step)
            elif 1000 <= current_time_step < 2000:
                transmission_events_2[mote].append(current_time_step)

        previously_polled_nodes = set(nodes_to_poll)

        for mote in nodes_to_poll:
            last_update_times[mote] = current_time_step

    valuable_sensor_df = pd.DataFrame(valuable_sensor_data)

    # Pad the MSE lists to the same length
    max_length = max(len(v) for v in mse_values.values())
    for key in mse_values:
        mse_values[key].extend([None] * (max_length - len(mse_values[key])))

    mse_df = pd.DataFrame(mse_values)

    return valuable_sensor_df, transmission_events_1, transmission_events_2, mse_df

results = {}
valuable_sensor_df_list = []
transmission_events_1_list = []
transmission_events_2_list = []
mse_df_list = []

for num_nodes_to_poll in num_nodes_to_poll_list:
    print(f"Running simulation for {num_nodes_to_poll} nodes to poll...")
    valuable_sensor_df, transmission_events_1, transmission_events_2, mse_df = run_simulation(num_nodes_to_poll)
    valuable_sensor_df_list.append((num_nodes_to_poll, valuable_sensor_df))
    transmission_events_1_list.append(transmission_events_1)
    transmission_events_2_list.append(transmission_events_2)
    mse_df_list.append(mse_df)
    print(f"Completed simulation for {num_nodes_to_poll} nodes to poll.")

# Calculate polling and transmission fractions
time_steps = len(pivot_df)
fw = {mote: polled_count[mote] / time_steps for mote in polled_count}
ft = {mote: transmitted_count[mote] / time_steps for mote in transmitted_count}

# Calculate the average lifetime of sensors in hours and years
average_lifetime_hours = np.mean([
    E_max / (ft[mote] * E_t + fw[mote] * (E_s + 3 * E_w) + (1 - fw[mote]) * E_0) for mote in polled_count
]) / 3600

average_lifetime_years = average_lifetime_hours / 8760  # Convert hours to years
print("Average sensor lifetime (years):", average_lifetime_years)

# Calculate the overall average MSE
overall_mse = mse_df_list[0].mean().mean()
print(f"Overall Average MSE: {overall_mse}")

# Plot the total transmission count for each node against the total simulation time steps
total_duration = len(pivot_df)
ratios = {int(mote.replace('mote', '')): transmitted_count[mote] / total_duration for mote in transmitted_count}

plt.figure(figsize=(8, 2))
plt.bar(ratios.keys(), ratios.values(), color='skyblue')
plt.xlabel('Node ID')
plt.ylabel('Transmission frequency')

# Set x-ticks at intervals of 10
ticks = list(range(0, 51, 10))
plt.xticks(ticks)

plt.show()
